In [1]:
import json
import numpy 
import os
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline  
import pyarrow
import sys
from tqdm import tqdm
import seaborn as sns
from datetime import date
from dotenv import load_dotenv
from sqlalchemy import create_engine
from os import path
from typing import List,Dict, Tuple
from collections import defaultdict
pd.set_option("display.max_columns", None)

load_dotenv(verbose=True)
BIGQUERY_CREDENTIALS_PATH = os.environ.get('BIGQUERY_CREDENTIALS_PATH')
engine = create_engine('bigquery://bespoke-financial/ProdMetrcData', credentials_path=os.path.expanduser(BIGQUERY_CREDENTIALS_PATH))

sys.path.append(path.realpath(path.join(os.getcwd(), "../core")))
sys.path.append(path.realpath(path.join(os.getcwd(), "../../src")))
sys.path.append(path.realpath(path.join(os.getcwd(), "../../scripts")))

import create_queries
import prepare_data

from bespoke.inventory.analysis.shared import download_util, inventory_types
from bespoke.inventory.analysis import active_inventory_util as util
from bespoke.inventory.analysis import inventory_valuations_util as valuations_util
from underwriting import msrp_band_analyzer

%load_ext autoreload
%autoreload 2

/Users/victoria/GitHub/venvs/bespoke/lib/python3.8/site-packages/pandas/compat/__init__.py:124: UserWarning: Could not import the lzma module. Your installed Python is incomplete. Attempting to use lzma compression will result in a RuntimeError.
  warnings.warn(msg)


In [2]:
COMPANY_IDENTIFIER_LIST = [
    'DL',
    'DW',
    'EMA',
    'EMM',
    'EMT',
    'EMF',
    'ST',
    'GRG',
    'EL',
    'VS',
]

In [3]:
mba = msrp_band_analyzer.MSRPBand()

In [4]:
mba.update_company_data(COMPANY_IDENTIFIER_LIST)


Verifying download summaries for license MR282376...
Earliest download summary: 2020-01-01
Latest download summary: 2022-04-11
Found bad download summary for license MR282376 on date 2022-03-31
Found bad download summary for license MR282376 on date 2022-03-24

Verifying download summaries for license MR283369...
Earliest download summary: 2020-01-01
Latest download summary: 2022-04-11
Found bad download summary for license MR283369 on date 2022-03-05

Verifying download summaries for license 402-00840...
Earliest download summary: 2020-01-01
Latest download summary: 2022-04-11

Verifying download summaries for license 403-01293...
Earliest download summary: 2020-01-01
Latest download summary: 2022-04-11

Verifying download summaries for license 402R-00545...
Earliest download summary: 2020-01-01
Latest download summary: 2022-04-11

Verifying download summaries for license 403R-01201...
Earliest download summary: 2020-01-01
Latest download summary: 2022-04-11

Verifying download summar

/Users/victoria/dev/platform/services/api-server/scripts/underwriting/msrp_band_analyzer.py:340: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['count_measure_from_product_name'][count_measure_non_na_index] = df['count_measure_from_product_name'][count_measure_non_na_index].apply(mba_util.extract_count_units)
/Users/victoria/dev/platform/services/api-server/scripts/underwriting/msrp_band_analyzer.py:345: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['extracted_units'][idx] = df[measure_column][idx].apply(mba_util.EXTRACTED_MEASUREMENT_COLUMNS[measure_column])
/Users/victoria/dev/platform/services/api-server/script

### Combining same product categories with different measurements ### 	


/Users/victoria/dev/platform/services/api-server/scripts/underwriting/msrp_band_analyzer.py:319: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[measure_ratio_column_name][includes_measurement] = self.unit_conversion_ratio(measurement, measurement_unit)
/Users/victoria/dev/platform/services/api-server/scripts/underwriting/msrp_band_analyzer.py:319: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[measure_ratio_column_name][includes_measurement] = self.unit_conversion_ratio(measurement, measurement_unit)
/Users/victoria/dev/platform/services/api-server/scripts/underwriting/msrp_band_analyzer.py:323: SettingWithCopyWarn

### Breaking down product names into different brands ### 	


/Users/victoria/dev/platform/services/api-server/scripts/underwriting/msrp_band_analyzer.py:357: UserWarning: This pattern has match groups. To actually get the groups, use str.extract.
  (df[product_name].str.contains("^(1 ml)") == False) &
/Users/victoria/dev/platform/services/api-server/scripts/underwriting/msrp_band_analyzer.py:357: UserWarning: This pattern has match groups. To actually get the groups, use str.extract.
  (df[product_name].str.contains("^(1 ml)") == False) &


### Available combined product category name in costs dataframe ### 	
['concentrate' 'infused' 'vape cartridge' 'edible' 'pre-roll infused'
 'shake/trim' 'flower' 'other concentrate' 'buds' 'pre-roll flower'
 'extract' 'pre-roll leaf' 'tincture' 'topical' 'capsule' 'shake'
 'wet whole plants' 'vape product' 'raw pre-rolls' 'seeds'
 'infused butter/oil' 'infused pre-rolls' 'kief' 'infused beverage'
 'shake/trim allocated for extraction' 'clone - cutting']
### Available combined product category name in sales dataframe ### 	
['other concentrate' 'edible' 'pre-roll flower' 'flower' 'buds'
 'concentrate' 'infused' 'shake/trim' 'extract' 'vape cartridge'
 'pre-roll infused' 'pre-roll leaf' 'topical' 'immature plants' 'capsule'
 'tincture' 'shake' 'raw pre-rolls' 'vape product'
 'shake/trim allocated for extraction' 'infused pre-rolls'
 'infused butter/oil' 'seeds' 'infused beverage' 'kief' 'clone - cutting']


In [5]:
###
df_sales = mba.company_sales_df.copy()

In [6]:
df_sales.head()

,license_number,receipt_number,rt_type,sales_customer_type,sales_datetime,sales_month,total_packages,rt_total_price,tx_type,tx_package_id,tx_package_label,tx_product_name,tx_product_category_name,tx_unit_of_measure,tx_quantity_sold,tx_total_price,tx_price_per_unit,date_in_month,bad_numbers_from_product_name,extracted_units,letter_gram_measure_from_product_name,count_measure_from_product_name,gram_measure_from_product_name,oz_measure_from_product_name,adjusted_tx_price_per_unit,measurement_ratio_vs_gram,combined_product_category,brand_breakable_by_dash_boolean,brands,brands_by_category
2356560,C10-0000169-LIC,0005181809,inactive,Consumer,2020-01-02 08:16:34+00:00,2020-01,2,33.76,inactive,1782428,1A4060300004D62000000186,VVS - Vape - 0.3g - Rose Gold - Indica - XXX OG,Other Concentrate (weight - each),Each,1.0,16.88,16.880000,2020-01,NaN,0.3,0.3g,NaN,NaN,NaN,56.266667,0.3,other concentrate,True,VVS,VVS (Other Concentrate (weight - each))
2336473,C10-0000169-LIC,0007521891,inactive,Consumer,2020-01-22 13:41:39+00:00,2020-01,5,36.58,inactive,1804087,1A4060300004B03000009761,LOL Edibles - Edible - 100mg - Doob Cube - Ras...,Edible (weight - each),Each,1.0,1.00,1.000000,2020-01,NaN,0.1,100mg,NaN,NaN,NaN,10.000000,0.1,edible,True,LOL Edibles,LOL Edibles (Edible (weight - each))
2336474,C10-0000169-LIC,0007521891,inactive,Consumer,2020-01-22 13:41:39+00:00,2020-01,5,36.58,inactive,1782339,1A4060300004D62000000057,Puffy Delivery - Pre Roll - 0.8g - Sativa - Fa...,Pre-Roll Flower,Each,1.0,8.68,8.680000,2020-01,NaN,0.8,0.8g,NaN,NaN,NaN,10.850000,0.8,pre-roll flower,True,Puffy Delivery,Puffy Delivery (Pre-Roll Flower)
2336475,C10-0000169-LIC,0007521891,inactive,Consumer,2020-01-22 13:41:39+00:00,2020-01,5,36.58,inactive,2111948,1A4060300004B03000010682,THC Design - Flower - 3.5g - Sativa - Crescendo,Flower,Grams,3.5,1.00,0.285714,2020-01,NaN,NaN,3.5g,NaN,NaN,NaN,0.285714,1.0,flower,True,THC Design,THC Design (Flower)
2336476,C10-0000169-LIC,0007521891,inactive,Consumer,2020-01-22 13:41:39+00:00,2020-01,5,36.58,inactive,1803899,1A4060300004B03000009717,LOL Edibles - Edible - 100mg - Doob Cube - Blu...,Edible (weight - each),Each,1.0,17.22,17.220000,2020-01,NaN,0.1,100mg,NaN,NaN,NaN,172.200000,0.1,edible,True,LOL Edibles,LOL Edibles (Edible (weight - each))


In [ ]:
###
df_sales.groupby(['tx_product_category_name','tx_unit_of_measure']).count().sort_values(by = 'tx_unit_of_measure',ascending = False)

# 1. uom - each:
- Using same UOM (ex. gram): comes in different quantity
- Using different UOM (ex. blend of gram and oz)

In [ ]:
## examples

In [ ]:
df_sales[(df_sales['tx_product_category_name'] == 'Other Concentrate (weight - each)')&(df_sales['tx_product_name'] == 'Wave - Cartridge - 0.5g - Hybrid - Lemon')]\
[['tx_product_category_name','tx_product_name','tx_unit_of_measure','tx_quantity_sold']].head()





In [ ]:
df_sales[(df_sales['tx_product_category_name'] == 'Other Concentrate (weight - each)')&(df_sales['tx_product_name'] == 'Thclear - 1g - Honey Pot - Skywalker OG')]\
[['tx_product_category_name','tx_product_name','tx_unit_of_measure','tx_quantity_sold']].head()





In [ ]:
## examples

In [ ]:

df_sales[(df_sales['tx_product_category_name'] == 'Edible (volume - each)')&(df_sales['tx_product_name'] == "Uncle Arnie's Iced Tea Lemonade Beverage 8oz Glass")]\
[['tx_product_category_name','tx_product_name','tx_unit_of_measure','tx_quantity_sold']].head()

In [ ]:

df_sales[(df_sales['tx_product_category_name'] == 'Edible (volume - each)')&(df_sales['tx_product_name'] == "Hapy - Edible - 100mg - Syrup Shot - Tropical Mix")]\
[['tx_product_category_name','tx_product_name','tx_unit_of_measure','tx_quantity_sold']].head()

# 2. how do we do regex

## 1. remove bad numbers

In [ ]:
df_sales[df_sales['bad_numbers_from_product_name'].notna()][['original_product_name','bad_numbers_from_product_name','tx_product_name']].head()

In [ ]:
df_sales[df_sales['bad_numbers_from_product_name'].notna()].shape[0] / df_sales.shape[0]

## 2. grab letter grams / gram / oz

In [ ]:
df_sales[df_sales['letter_gram_measure_from_product_name'].notna()][['tx_product_name','letter_gram_measure_from_product_name',]].head()




In [ ]:
df_sales[df_sales['letter_gram_measure_from_product_name'].notna()].shape[0] / df_sales.shape[0]

In [ ]:

df_sales[(df_sales['gram_measure_from_product_name'].notna()) & (df_sales['gram_measure_from_product_name'] == 'gram')][['tx_product_name','gram_measure_from_product_name']]\
.head()




In [ ]:
df_sales[df_sales['gram_measure_from_product_name'].notna()].shape[0] / df_sales.shape[0]

In [ ]:


df_sales[(df_sales['oz_measure_from_product_name'].notna())][['tx_product_name','oz_measure_from_product_name']]\
.head()




In [ ]:
df_sales[df_sales['oz_measure_from_product_name'].notna()].shape[0] / df_sales.shape[0]

# 3. combine 

In [ ]:
df_sales[df_sales['letter_gram_measure_from_product_name'].notna()][['tx_product_name','letter_gram_measure_from_product_name','extracted_units']].head()




In [ ]:

df_sales[(df_sales['gram_measure_from_product_name'].notna()) & (df_sales['gram_measure_from_product_name'] == 'gram')][['tx_product_name','gram_measure_from_product_name','extracted_units']]\
.head()




In [ ]:


df_sales[(df_sales['oz_measure_from_product_name'].notna())]




# 4. coverage

In [ ]:
df_sales

In [ ]:
##
df_coverage = pd.DataFrame()
for p in df_sales[df_sales['tx_unit_of_measure'] != 'Grams']['tx_product_category_name'].unique():
    print(p)
    df_temp = pd.DataFrame([mba.company_sales_df[(mba.company_sales_df['tx_product_category_name'] == p)&(mba.company_sales_df['extracted_units'].notna())].shape[0]\
    / mba.company_sales_df[(mba.company_sales_df['tx_product_category_name'] == p)].shape[0]])
    df_temp['category'] = p
    df_coverage = df_coverage.append(df_temp)




# 5. sanity checks

In [ ]:
df_sales[df_sales['tx_product_category_name'] == 'Flower (packaged quarter - each)']

In [ ]:
###
#mba.company_costs_df.groupby(['product_category_name','shipped_unit_of_measure']).count()

In [ ]:
###
#mba.company_sales_df[mba.company_sales_df['tx_product_category_name'] == 'Concentrate']

In [ ]:
pd.DataFrame([mba.company_sales_df[(mba.company_sales_df['tx_product_category_name'] == 'Flower (packaged quarter - each)')&(mba.company_sales_df['extracted_units'].notna())].shape[0]\
/ mba.company_sales_df[(mba.company_sales_df['tx_product_category_name'] == 'Flower (packaged quarter - each)')].shape[0]])





In [ ]:

mba.company_sales_df[mba.company_sales_df['tx_product_category_name'] == 'Flower (packaged quarter - each)'].extracted_units.value_counts(normalize = False)

In [ ]:
mba.company_sales_df[(mba.company_sales_df['tx_product_category_name'] == 'Capsule (weight - each)')]



In [ ]:
##
df = pd.DataFrame()
for p in mba.company_sales_df['tx_product_category_name'].unique():
    print(p)
    df_temp = mba.company_sales_df[mba.company_sales_df['tx_product_category_name'] == p].extracted_units.value_counts(normalize = True).to_frame().sort_values(by ='extracted_units',ascending = False )
    df_temp['category'] = p
    df = df.append(df_temp)




In [ ]:
##
df_coverage = pd.DataFrame()
for p in mba.company_sales_df['tx_product_category_name'].unique():
    print(p)
    df_temp = pd.DataFrame([mba.company_sales_df[(mba.company_sales_df['tx_product_category_name'] == p)&(mba.company_sales_df['extracted_units'].notna())].shape[0]\
    / mba.company_sales_df[(mba.company_sales_df['tx_product_category_name'] == p)].shape[0]])
    df_temp['category'] = p
    df_coverage = df_coverage.append(df_temp)




In [ ]:
df_coverage

In [ ]:
#df.to_csv('dominant_unit.csv')

In [ ]:
mba.company_sales_df[(mba.company_sales_df['tx_product_category_name'] == 'Concentrate (Each)')&(mba.company_sales_df['extracted_units'].isna())]



In [ ]:
###
mba.company_sales_df[(mba.company_sales_df.tx_product_category_name == 'Flower (packaged quarter - each)')&(mba.company_sales_df['extracted_units'].notna())][['adjusted_tx_price_per_unit','sales_month']].groupby(['sales_month']).median()



In [ ]:
mba.company_sales_df.adjusted_tx_price_per_unit.isna().sum()

In [ ]:
mba.company_sales_df.adjusted_tx_price_per_unit.shape

In [ ]:
temp = mba.company_sales_df.copy()

In [ ]:
(temp.letter_gram_measure_from_product_name.isna() == False).sum()

In [ ]:
temp.head()

In [ ]:
((temp.tx_unit_of_measure == 'Each') & (temp.extracted_units.isna() == True) & (temp.measurement_ratio_vs_gram == 1)).sum()

In [ ]:
## Coverage %
1 - (299024/1405912)

In [ ]:
mba.run_analysis('tx_product_category_name', 'Pre-Roll Flower', 'S')

In [ ]:
mba.run_analysis('product_category_name', 'Pre-Roll Flower', 'C')

In [ ]:
mba.run_analysis('combined_product_category', 'flower', 'S')

In [ ]:
mba.run_analysis('combined_product_category', 'flower', 'C')

In [ ]:
###
#mba.company_sales_df[mba.company_sales_df.tx_product_category_name == 'Flower (packaged quarter - each)'][['adjusted_tx_price_per_unit','sales_month']].groupby(['sales_month']).mean()



In [ ]:
combined_median_price = mba.company_sales_df[mba.company_sales_df.combined_product_category == 'flower'].adjusted_tx_price_per_unit.median()

eighth_median_price = mba.company_sales_df[mba.company_sales_df.tx_product_category_name == 'Flower (packaged eighth - each)'].tx_price_per_unit.median()
quarter_median_price = mba.company_sales_df[mba.company_sales_df.tx_product_category_name == 'Flower (packaged quarter - each)'].tx_price_per_unit.median()
half_once_median_price = mba.company_sales_df[mba.company_sales_df.tx_product_category_name == 'Flower (packaged half ounce - each)'].tx_price_per_unit.median()
once_median_price = mba.company_sales_df[mba.company_sales_df.tx_product_category_name == 'Flower (packaged ounce - each)'].tx_price_per_unit.median()

In [ ]:
combined_median_price_flower_non_labeled = mba.company_sales_df[mba.company_sales_df.tx_product_category_name == 'Flower'].adjusted_tx_price_per_unit.median()


In [ ]:
combined_median_price

In [ ]:
combined_median_price_flower_non_labeled

In [ ]:
eighth_median_price / 3.5

In [ ]:
quarter_median_price / 7

In [ ]:
half_once_median_price / 14

In [ ]:
once_median_price / 28

In [ ]:
mba.company_sales_df[mba.company_sales_df.tx_product_category_name.str.contains('Flower', case = False)][['tx_price_per_unit', 'adjusted_tx_price_per_unit', 'tx_product_category_name']].groupby('tx_product_category_name').describe()

We see that adjusted_price is overall on a smaller scale than tx_price_per_unit due to units being converted down to grams. 

In [ ]:
mba.output_time_series_metadata()

In [ ]:
mba.save()

In [ ]:
import pickle
picklefile = open('msrp_band_analyzer_training_object', 'rb')
old_mba = pickle.load(picklefile)

In [ ]:
old_mba['company_sales_df'].head()

In [ ]:
new_mba = msrp_band_analyzer.MSRPBand(company_costs_df = old_mba['company_costs_df'], company_sales_df = old_mba['company_sales_df'])

In [ ]:
new_mba.company_sales_df.head()

In [ ]:
mba.msrp_summary_table_by_time

# edible

In [ ]:
df_sales.head()

In [ ]:
#df_sales[df_sales['count_measure_from_product_name'].notna()]['tx_product_category_name'].unique()

In [ ]:
# for p in df_sales['tx_product_category_name'].unique():
#     print(p)
#     if p in ['Shake/Trim (by strain)','Shake/Trim','Infused Butter/Oil (weight - each)','Infused Butter/Oil (volume - each)',\
#             'Shake/Trim allocated for extraction']:
#         continue
#     else:
#         temp = df_sales[df_sales['tx_product_category_name'] == p]
#         temp_gb = temp[['letter_gram_measure_from_product_name','tx_price_per_unit']].groupby(['letter_gram_measure_from_product_name']).count().\
#         reset_index()['letter_gram_measure_from_product_name'].str.lower()
#         #print(temp_gb)
#         temp_gb.to_csv(str(p) + '.csv')

In [7]:
data= df_sales[df_sales['tx_product_category_name'] == 'Pre-Roll Infused']




In [8]:
data.shape[0]

118485

In [9]:
#letter gram
data\
[['letter_gram_measure_from_product_name','tx_price_per_unit']].groupby(['letter_gram_measure_from_product_name']).count().reset_index()



,letter_gram_measure_from_product_name,tx_price_per_unit
0,.5 g,47
1,.5G,19
2,.5g,6257
3,.6g,9
4,.7 g,49
5,.7g,687
6,0.35g,183
7,0.3g,3
8,0.59g,56
9,0.5G,86


In [10]:
#oz
data\
[['oz_measure_from_product_name','tx_price_per_unit']].groupby(['oz_measure_from_product_name']).count().reset_index()



,oz_measure_from_product_name,tx_price_per_unit


In [11]:
# count/piece
data\
[['count_measure_from_product_name','tx_price_per_unit']].groupby(['count_measure_from_product_name']).count().reset_index()



,count_measure_from_product_name,tx_price_per_unit
0,1.0,3
1,2.0,153
2,3.0,25
3,4.0,707
4,5.0,827
5,6.0,1241
6,7.0,179
7,12.0,668
8,15.0,231
9,16.0,189


In [12]:
data\
[['letter_gram_measure_from_product_name','count_measure_from_product_name','tx_price_per_unit']].groupby(['letter_gram_measure_from_product_name','count_measure_from_product_name']).count().reset_index()



,letter_gram_measure_from_product_name,count_measure_from_product_name,tx_price_per_unit
0,.5g,1.0,3
1,.5g,3.0,13
2,.5g,5.0,773
3,.5g,6.0,288
4,.5g,12.0,83
5,.6g,5.0,9
6,0.5g,2.0,15
7,0.5g,5.0,14
8,0.5g,6.0,953
9,0.5g,15.0,231


In [13]:
data\
[['letter_gram_measure_from_product_name','oz_measure_from_product_name','tx_price_per_unit']].groupby(['letter_gram_measure_from_product_name','oz_measure_from_product_name']).count().reset_index()



,letter_gram_measure_from_product_name,oz_measure_from_product_name,tx_price_per_unit


In [14]:
data\
[['count_measure_from_product_name','oz_measure_from_product_name','tx_price_per_unit']].groupby(['count_measure_from_product_name','oz_measure_from_product_name']).count().reset_index()



,count_measure_from_product_name,oz_measure_from_product_name,tx_price_per_unit


In [15]:
data[data['oz_measure_from_product_name'].notna()]['tx_product_name'].unique()




array([], dtype=object)

In [16]:
data[data['count_measure_from_product_name'].notna()]['tx_product_name'].unique()




array([' Space Coyote x Utopia Infused Preroll 1.0g Indica (24ct)',
       ' Space Coyote x Utopia Infused Preroll 1.0g Sativa (24ct)',
       'Highrize Venom OG HI 4pk', 'Highrize 12pk mimosa',
       'High Rize HI 12pk Tins Moon Drop',
       'highrize 12pk infused wedding cake',
       'Highrize Hash Infused Cherry AK 4pk',
       'Highrize Hash Infused 12pk Cookies',
       'Highrize Hash Infused 12pk White Widow',
       'Highrize Hash Infused 12pk Skunk Dawg',
       ' Garden Society Infused Preroll Rosettes Sativa Sample Case (2ct)',
       'Highrize Peanut Butter Breath HI 4pk Tube',
       'Highrize Peanut Butter Breath HI 12pk Tins',
       'CLSICS Gorilla Glue x Oak OG Rosin (2.8g) Infused 4ct Pre-roll',
       'CLSICS Peanut Butter Breath x Ice Queen Rosin (2.8g) Infused 4ct Pre-roll',
       ' Garden Society Infused Preroll Rosettes .5g Hash Indica 4-Pack 2.0g (12ct)',
       ' Space Coyote Infused Preroll 0.5g Sativa Hash 5-Pack (15ct)',
       ' Space Coyote Infused Prer

In [ ]:
data[data['letter_gram_measure_from_product_name'] == '3.5g']['tx_price_per_unit'].describe()



In [17]:
data[data['count_measure_from_product_name'].isna()]['tx_price_per_unit'].describe()



count    113274.000000
mean         19.382685
std          11.284043
min           0.011667
25%          10.920000
50%          17.060000
75%          26.020000
max         202.000000
Name: tx_price_per_unit, dtype: float64

In [ ]:
data[data['count_measure_from_product_name'] == 20]

In [ ]:
data[data['tx_product_name'] == ' Raw Garden Cartridge 1.0g Indica Fire Walker (10ct)']



In [ ]:
df_sales[df_sales['tx_product_category_name'] == 'Infused (edible)']\
[['count_measure_from_product_name','letter_gram_measure_from_product_name','tx_price_per_unit']].groupby(['count_measure_from_product_name','letter_gram_measure_from_product_name']).count().reset_index()




In [ ]:
edible = df_sales[df_sales['tx_product_category_name'] == 'Shake (Packaged Eighth - each)']






In [ ]:
edible[edible['letter_gram_measure_from_product_name'].notna()]['tx_product_name'].unique()

In [ ]:
pd.DataFrame(edible[edible['letter_gram_measure_from_product_name'].notna()]['tx_product_name'].unique())







In [ ]:
edible[['oz_measure_from_product_name','tx_price_per_unit']].groupby(['oz_measure_from_product_name']).count()




# Testing

In [ ]:
from underwriting import msrp_band_analyzer_testing


In [ ]:
result = msrp_band_analyzer_testing.run_test_set_analysis('product_category_name', 'Flower', 'C')

In [ ]:
result[0]

In [ ]:
result[1]

In [ ]:
result = run_test_set_analysis('tx_product_category_name', 'Pre-Roll Flower', 'S')